# Questions by Daniel Majer

### Gráfico de Barras

What's the range of ages in each race group, when most probable that women can be affected by breast cancer based on this data set?

Poner solo el gráfico de barras sin la normalización (líneas) eso será lo siguiente.
https://www.kaggle.com/code/koustavghosh149/data-visualization-using-breast-cancer-dataset?scriptVersionId=103044427&cellId=17

### Gráfico de Barras con Normalización
In those ages which suffer the most from breast cancer, what are the frequencies of the 6th stage categories?

https://www.kaggle.com/code/koustavghosh149/data-visualization-using-breast-cancer-dataset?scriptVersionId=103044427&cellId=17


### Scatter plots y rectas de regresión
Edad en la que es más probable que vivan o mueran los pacientes.

Boxplot
mortalidad y supervivencia basado en el número de meses que ha estado el tumor dentro del cuerpo.
sns.boxplot('Survival Months', 'Status', data=df)

Conclusión
tiendes a pensar que mientras más tiempo está el tumor en tu cuerpo menos probabilidades de vivir tienes, pero es al revés.
